In [ ]:
# Cell 1: Set up the path
import sys
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()
# Get the SRC_PATH from the environment variable
src_path = os.getenv('SRC_PATH')

print(f"SRC_PATH: {src_path}")

if src_path:
    # Add the src path to sys.path
    sys.path.append(os.path.abspath(src_path))

SRC_PATH: D:\CODE\de_projects\finance_hub\src


## Connector

### VnStock

In [2]:
from pipeline.commons.connectors.vnstock_lib import VnstockLibConnector

D:\CODE\de_projects\finance_hub\src is already in sys.path


In [3]:
connector = VnstockLibConnector(init_symbol='MBB')

2024-10-31 00:52:06,232 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Connecting to VnstockLib: Symbol: MBB, Source: VCI
Connected to VnstockLib.


In [4]:
connector.list_all_symbols()

,ticker,organ_name
0,A32,Công ty Cổ phần 32
1,AAA,Công ty Cổ phần Nhựa An Phát Xanh
2,AAM,Công ty Cổ Phần Thủy Sản MeKong
3,AAS,Công ty Cổ phần Chứng khoán SmartInvest
4,AAT,Công ty Cổ phần Tập đoàn Tiên Sơn Thanh Hóa
...,...,...
1593,XPH,Công ty Cổ phần Xà phòng Hà Nội
1594,YBC,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái
1595,YBM,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái
1596,YEG,Công ty Cổ phần Tập đoàn Yeah1


In [5]:
full_info = connector.get_full_company_info()

Getting full company info


In [6]:
# connector.list_all_symbols()

In [7]:
# connector.get_symbols_by_exchange()

In [8]:
# connector.get_full_company_info()

In [9]:
connector.get_stock_quote_history_df()

2024-10-31 00:52:08,300 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Fetching history for symbol: MBB


,time,open,high,low,close,volume,symbol,loaded_timestamp
0,2011-11-01,2.55,2.55,2.42,2.42,2719290,MBB,2024-10-30 17:52:08.853993+00:00
1,2011-11-02,2.39,2.42,2.32,2.32,1255220,MBB,2024-10-30 17:52:08.853993+00:00
2,2011-11-03,2.28,2.32,2.21,2.30,1193910,MBB,2024-10-30 17:52:08.853993+00:00
3,2011-11-04,2.32,2.32,2.27,2.27,483860,MBB,2024-10-30 17:52:08.853993+00:00
4,2011-11-07,2.27,2.27,2.16,2.16,1158750,MBB,2024-10-30 17:52:08.853993+00:00
...,...,...,...,...,...,...,...,...
3238,2024-10-21,25.75,25.90,25.55,25.55,9410500,MBB,2024-10-30 17:52:08.853993+00:00
3239,2024-10-22,25.50,25.65,25.15,25.25,20327200,MBB,2024-10-30 17:52:08.853993+00:00
3240,2024-10-23,25.25,25.35,25.15,25.25,11634500,MBB,2024-10-30 17:52:08.853993+00:00
3241,2024-10-24,25.25,25.40,24.70,24.70,16246700,MBB,2024-10-30 17:52:08.853993+00:00


In [10]:
# connector.get_stock_quote_history_dict()

### S3

In [14]:
import os
from dotenv import load_dotenv
import boto3
load_dotenv()

True

In [15]:
import pandas as pd
from pipeline.commons.connectors.s3 import S3BucketConnector

# Configuration parameters for MinIO
config = {
    "aws_access_key_id": os.getenv('MINIO_ACCESS_KEY'),
    "aws_secret_access_key": os.getenv('MINIO_SECRET_KEY'),
    "endpoint_url": os.getenv('MINIO_URL'),
    "bucket": "vnstock3", 
}
bucket_name = config["bucket"]

# Create an instance of the S3 connector
s3_connector = S3BucketConnector(config, bucket_name)

In [13]:
# Example DataFrame to write
data = {
    "column1": [1, 2, 3],
    "column2": ["A", "B", "C"]
}
df = pd.DataFrame(data)

# Write DataFrame to S3 as JSON
key_json = "test_connector/example1.json"
s3_connector.write_df_to_s3(df, key_json, "json")

-----Writing file to http://localhost:9000 vnstock3 test_connector/example1.json


True

In [14]:
# List files in a prefix
prefix = "test_connector/"
files = s3_connector.list_file_in_prefix(prefix)
print("Files in prefix:", files)

Files in prefix: ['test_connector/example1.json']


In [15]:
s3_connector._bucket

s3.Bucket(name='vnstock3')

In [16]:
s3_connector._bucket.objects.filter(Prefix='data')

s3.Bucket.objectsCollection(s3.Bucket(name='vnstock3'), s3.ObjectSummary)

In [17]:
from io import StringIO

# Sample buffer with some JSON content
buffer = StringIO('{"symbol": "AAPL", "max_timestamp": "2024-10-27T12:00:00"}')

# Upload the buffer to S3
s3_connector._put_buffer(buffer, 'test_connector/metadata/symbols_meta.json')


-----Writing file to http://localhost:9000 vnstock3 metadata/symbols_meta.json


True

In [18]:
import json
from io import BytesIO

# Sample data to be saved as JSON
data = {"symbol": "AAPL", "max_timestamp": "2024-10-27T12:00:00"}

# Convert data to JSON and write to BytesIO
json_content = json.dumps(data)
buffer = BytesIO(json_content.encode('utf-8'))  # Encode to bytes

# Now upload the buffer
s3_connector._put_buffer(buffer, 'test_connector/metadata/symbols_meta_bytes.json')

-----Writing file to http://localhost:9000 vnstock3 metadata/symbols_meta_bytes.json


True

In [31]:
s3_connector._put_object(buffer, 'test_connector/metadata/symbols_meta_object_app.json', 'application/json')

True

## Extractor

In [2]:
import time
import pandas as pd
from pipeline.extract.vnstock_lib import VnstockLibExtractor
from pipeline.commons.constants import DateTimeFormat

extractor = VnstockLibExtractor()

D:\CODE\de_projects\finance_hub\src is already in sys.path
D:\CODE\de_projects\finance_hub\src is already in sys.path


2024-11-01 10:56:25,592 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Connecting to VnstockLib: Symbol: VNN, Source: VCI
Connected to VnstockLib.


In [3]:
df_all = extractor.get_all_stock_quote_histories_df(symbols=['MBB', 'VCB', 'YTC'])

2024-11-01 10:56:26,039 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Processing batch 1: ['MBB', 'VCB', 'YTC']
Fetching history for symbol: MBB


2024-11-01 10:56:26,670 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Fetching history for symbol: VCB


2024-11-01 10:56:27,130 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Fetching history for symbol: YTC


In [4]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8881 entries, 0 to 8880
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   time              8881 non-null   datetime64[ns]     
 1   open              8881 non-null   float64            
 2   high              8881 non-null   float64            
 3   low               8881 non-null   float64            
 4   close             8881 non-null   float64            
 5   volume            8881 non-null   int64              
 6   symbol            8881 non-null   object             
 7   loaded_timestamp  8881 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 555.2+ KB


In [7]:
max_ts = df_all['loaded_timestamp'].groupby(df_all['symbol']).max()
max_ts

symbol
MBB   2024-10-31 03:44:33.954043+00:00
VCB   2024-10-31 03:44:34.388914+00:00
VIC   2024-10-31 03:44:34.844432+00:00
VNM   2024-10-31 03:44:35.277768+00:00
Name: loaded_timestamp, dtype: datetime64[ns, UTC]

In [19]:
metadata_file = 'hub/metadata/stock/stock_quotes_state.json'

for symbol, ts in max_ts.items():
    print(symbol, ts)
    s3_connector.update_symbol_meta_timestamp(symbol, ts, metadata_file)

MBB 2024-10-31 03:44:33.954043+00:00
{'MBB': {'max_timestamp': '2024-10-31T03:44:33'}}
VCB 2024-10-31 03:44:34.388914+00:00
{'MBB': {'max_timestamp': '2024-10-31T03:44:33'}, 'VCB': {'max_timestamp': '2024-10-31T03:44:34'}}
VIC 2024-10-31 03:44:34.844432+00:00
{'MBB': {'max_timestamp': '2024-10-31T03:44:33'}, 'VCB': {'max_timestamp': '2024-10-31T03:44:34'}, 'VIC': {'max_timestamp': '2024-10-31T03:44:34'}}
VNM 2024-10-31 03:44:35.277768+00:00
{'MBB': {'max_timestamp': '2024-10-31T03:44:33'}, 'VCB': {'max_timestamp': '2024-10-31T03:44:34'}, 'VIC': {'max_timestamp': '2024-10-31T03:44:34'}, 'VNM': {'max_timestamp': '2024-10-31T03:44:35'}}


## Transformer

## Loader